In [167]:
import pandas as pd
import numpy as np
import os

In [168]:
model_file = os.listdir('../data/')
model_file = [i for i in model_file if 'model_result' in i]

In [257]:
df_model_list = []

for file_name in model_file:
    file_location = '../data/' + file_name
    ticker_name = file_name.split('_')[0]
    df = pd.read_csv(file_location)

    metrics_name = ['test_sharpe','test_rmse','test_spearmanr','train_cagr','train_sharpe_ratio','calmar']
    model = ['ridge','rf','gb']
    df['ticker_name'] = ticker_name
    for i in metrics_name:
        df[i + '_rank'] = df.groupby("model_name")[i].rank(ascending=False)
    df_model_list.append(df)
df_model = pd.concat(df_model_list)
df_model = df_model.replace([np.inf, -np.inf], np.nan,)

In [209]:
## we have 19 indicators per model, including one base model
## df_model has 1140 rows, 19 * 20(tickers) * 3(model)
print(len(list(set(df_model['col']))))
list(set(df_model['col']))


19


['RSI_25',
 'CMMA_10_252',
 'MACD_10_100_5',
 'RSI_20_ta',
 'MADIFF_10_100',
 'MADIFF_10_100_0',
 'LINTRND_10',
 'FTI_BP',
 'FTI_LP',
 'base',
 'RSI_20',
 'RSI_25_ta',
 'STO_20_1',
 'FTI_BF',
 'PR_INT_0',
 'DT_RSI_2_20',
 'ENT_4_16',
 'ENT_2_10',
 'PR_INT_20']

In [258]:
df_model.columns

Index(['col', 'model_name', 'test_sharpe', 'test_rmse', 'test_spearmanr',
       'train_cagr', 'train_sharpe_ratio', 'calmar', 'ticker_name',
       'test_sharpe_rank', 'test_rmse_rank', 'test_spearmanr_rank',
       'train_cagr_rank', 'train_sharpe_ratio_rank', 'calmar_rank'],
      dtype='object')

In [259]:
req_col = ['ticker_name','col','model_name','test_spearmanr_rank','test_sharpe_rank','train_cagr','train_sharpe_ratio','calmar','train_cagr_rank','train_sharpe_ratio_rank','calmar_rank']
df_base_result = df_model[df_model['col'] == 'base'][req_col]
metrics_col = ['test_spearmanr_rank','test_sharpe_rank','train_cagr_rank','train_sharpe_ratio_rank','calmar_rank']

for metrics_name in metrics_col:
    df_base_result[metrics_name + '_level'] = pd.cut(df_base_result[metrics_name],[0,2,5,10,20], labels = ['level0-2','level2-5','level5-10','level10-20'])

In [260]:
from collections import Counter
print('test spearmanr rank counts')
print(Counter(df_base_result['test_spearmanr_rank_level']))
print('test sharpe rank counts')
print(Counter(df_base_result['test_sharpe_rank_level']))
print('train cagr rank counts')
print(Counter(df_base_result['train_cagr_rank_level']))
print('train sharpe ratio rank counts')
print(Counter(df_base_result['train_sharpe_ratio_rank_level']))
print('calmar rank level counts')
print(Counter(df_base_result['calmar_rank_level']))


test spearmanr rank counts
Counter({'level10-20': 52, 'level5-10': 8})
test sharpe rank counts
Counter({'level10-20': 24, 'level5-10': 21, 'level2-5': 10, 'level0-2': 5})
train cagr rank counts
Counter({'level10-20': 41, 'level5-10': 15, 'level2-5': 4})
train sharpe ratio rank counts
Counter({'level10-20': 41, 'level5-10': 15, 'level2-5': 4})
calmar rank level counts
Counter({'level10-20': 39, 'level5-10': 10, 'level2-5': 8, 'level0-2': 3})


## Baseline model VS Indicator model (regarding on spearmanr and rmse)

In [287]:
df_base = df_model[df_model['col'] == 'base'][['model_name','ticker_name','test_spearmanr','test_rmse']]
df_ind = df_model[df_model['col'] != 'base'][['col','model_name','ticker_name','test_spearmanr','test_rmse']]
df_base.columns = ['model_name','ticker_name','test_spearmanr_base','test_rmse_base']
df_comparison = df_ind.merge(df_base, left_on=['model_name','ticker_name'], right_on=['model_name','ticker_name'], how = 'left')
df_comparison

,col,model_name,ticker_name,test_spearmanr,test_rmse,test_spearmanr_base,test_rmse_base
0,RSI_20,ridge,AAPL,0.046232,-0.018597,0.044724,-0.018606
1,RSI_20_ta,ridge,AAPL,0.040550,-0.018622,0.044724,-0.018606
2,RSI_25,ridge,AAPL,0.044396,-0.018605,0.044724,-0.018606
3,RSI_25_ta,ridge,AAPL,0.042790,-0.018614,0.044724,-0.018606
4,DT_RSI_2_20,ridge,AAPL,0.214286,-0.018150,0.044724,-0.018606
...,...,...,...,...,...,...,...
1075,ENT_2_10,gb,DIS,0.033924,-0.015425,0.026543,-0.015407
1076,ENT_4_16,gb,DIS,0.018475,-0.015690,0.026543,-0.015407
1077,FTI_LP,gb,DIS,0.026368,-0.016303,0.026543,-0.015407
1078,FTI_BP,gb,DIS,0.029313,-0.015299,0.026543,-0.015407


In [288]:
comparison_metrics = ['test_spearmanr', 'test_rmse']
print('The % that indicators outforms the baseline model')
for comparison_m in comparison_metrics:
    base_m = comparison_m + '_base'
    df_comparison[comparison_m + '_better'] = (df_comparison[comparison_m] > df_comparison[base_m]).astype(int)
    print(comparison_m, np.sum(df_comparison[comparison_m + '_better'] )/ df_comparison[comparison_m].count())

The % that indicators outforms the baseline model
test_spearmanr 0.7027777777777777
test_rmse 0.4925925925925926


In [289]:


df_model_comparsion = df_comparison.groupby(['model_name'])[['test_spearmanr_better','test_rmse_better']].agg(lambda x: x.sum()/x.count())
df_model_comparsion

,test_spearmanr_better,test_rmse_better
model_name,,
gb,0.705556,0.350000
rf,0.727778,0.566667
ridge,0.675000,0.561111


In [291]:
df_indicator_comparison = df_comparison.groupby(['col'])[['test_spearmanr_better','test_rmse_better']].agg(lambda x: x.sum()/x.count())
df_indicator_comparison.sort_values(['test_spearmanr_better','test_rmse_better'],ascending=False)

,test_spearmanr_better,test_rmse_better
col,,
PR_INT_0,1.000000,0.950000
DT_RSI_2_20,1.000000,0.900000
CMMA_10_252,1.000000,0.866667
PR_INT_20,1.000000,0.733333
RSI_20,0.950000,0.716667
MACD_10_100_5,0.916667,0.700000
RSI_25,0.916667,0.683333
LINTRND_10,0.866667,0.516667
STO_20_1,0.816667,0.666667


## Timothy Model VS Talib Indicator (on test spearmanr, test rmse)

In [293]:
df_rsi = df_model[df_model['col'].isin(['RSI_20', 'RSI_20_ta','RSI_25', 'RSI_25_ta'])]
df_rsi = df_rsi[['ticker_name','col','test_spearmanr','test_rmse']].set_index(['ticker_name','col'])\
    .groupby(level=['col','ticker_name']) \
    .apply(lambda g: g.apply('mean')) \
    .unstack(level='col')\
    .reset_index()
df_rsi.columns = ['_'.join(i).rstrip('_') for i in df_rsi.columns]
df_rsi.head(5)

,ticker_name,test_spearmanr_RSI_20,test_spearmanr_RSI_20_ta,test_spearmanr_RSI_25,test_spearmanr_RSI_25_ta,test_rmse_RSI_20,test_rmse_RSI_20_ta,test_rmse_RSI_25,test_rmse_RSI_25_ta
0,AAPL,0.065734,0.033692,0.051148,0.033044,-0.019399,-0.019421,-0.019372,-0.019458
1,AMZN,0.066271,0.020626,0.057158,0.024408,-0.019991,-0.019996,-0.019913,-0.020057
2,BA,0.029651,-0.004160,0.013822,-0.003725,-0.021015,-0.021160,-0.021050,-0.021147
3,BAC,0.061831,0.023200,0.045896,0.023160,-0.021048,-0.020994,-0.021327,-0.020956
4,CAT,0.008246,0.000555,0.008247,-0.007935,-0.018037,-0.017974,-0.018086,-0.018085


In [295]:
timothy_columns = ['test_spearmanr_RSI_20', 
       'test_spearmanr_RSI_25', 
       'test_rmse_RSI_20',
       'test_rmse_RSI_25'
       ]
rsi_result_dict = []
for i in list(range(len(timothy_columns))):
    tim_colname = timothy_columns[i]
    ta_colname = tim_colname + '_ta'
    df_rsi[tim_colname + '_better'] = (df_rsi[tim_colname] > df_rsi[ta_colname]).astype(int)
    df_rsi[tim_colname + '_ratio'] = (df_rsi[tim_colname] / df_rsi[ta_colname]).astype(float)

    rsi_result_dict.append({'indicator': tim_colname, '# tim > ta': np.sum(df_rsi[tim_colname + '_better']),
    '% tim > ta to entire cases': np.sum(df_rsi[tim_colname + '_better'])  /df_rsi.shape[0],
    'avg % tim / ta': round((np.mean(df_rsi[tim_colname + '_ratio']) * 100),2)
     })

pd.DataFrame(rsi_result_dict)


,indicator,# tim > ta,% tim > ta to entire cases,avg % tim / ta
0,test_spearmanr_RSI_20,20,1.00,489.46
1,test_spearmanr_RSI_25,18,0.90,359.20
2,test_rmse_RSI_20,16,0.80,98.46
3,test_rmse_RSI_25,13,0.65,98.75


## Base model VS indicators(on the CAGR and Sharpe ratio)

In [208]:
df_base = df_model[df_model['col'] == 'base'][['model_name','ticker_name','train_cagr','train_sharpe_ratio','calmar']]
df_ind = df_model[df_model['col'] != 'base'][['col','model_name','ticker_name','train_cagr','train_sharpe_ratio','calmar']]
df_base.columns = ['model_name','ticker_name','train_cagr_base','train_sharpe_ratio_base','calmar_base']
df_comparison = df_ind.merge(df_base, left_on=['model_name','ticker_name'], right_on=['model_name','ticker_name'], how = 'left')
df_comparison


,col,model_name,ticker_name,train_cagr,train_sharpe_ratio,calmar,train_cagr_base,train_sharpe_ratio_base,calmar_base
0,RSI_20,ridge,AAPL,0.480617,1.486366,1.832415,0.469319,1.459971,1.663959
1,RSI_20_ta,ridge,AAPL,0.481233,1.487786,1.753116,0.469319,1.459971,1.663959
2,RSI_25,ridge,AAPL,0.469147,1.459474,1.652338,0.469319,1.459971,1.663959
3,RSI_25_ta,ridge,AAPL,0.479775,1.484413,1.545570,0.469319,1.459971,1.663959
4,DT_RSI_2_20,ridge,AAPL,1.650218,3.542123,6.280083,0.469319,1.459971,1.663959
...,...,...,...,...,...,...,...,...,...
1075,ENT_2_10,gb,DIS,12.273549,14.573869,NaN,12.272935,14.573394,69243.249349
1076,ENT_4_16,gb,DIS,12.273328,14.573698,122927.936580,12.272935,14.573394,69243.249349
1077,FTI_LP,gb,DIS,12.273549,14.573869,NaN,12.272935,14.573394,69243.249349
1078,FTI_BP,gb,DIS,12.273328,14.573698,122927.936580,12.272935,14.573394,69243.249349


In [227]:
comparison_metrics = ['train_cagr','train_sharpe_ratio','calmar']
print('The % that indicators outforms the baseline model')
for comparison_m in comparison_metrics:
    base_m = comparison_m + '_base'
    df_comparison[comparison_m + '_better'] = (df_comparison[comparison_m] > df_comparison[base_m]).astype(int)
    print(comparison_m, np.sum(df_comparison[comparison_m + '_better'] )/ df_comparison[comparison_m].count())

df_comparison.loc[df_comparison['calmar'].isna(),'calmar_better'] = np.nan

The % that indicators outforms the baseline model
train_cagr 0.6361111111111111
train_sharpe_ratio 0.6361111111111111
calmar 0.5985915492957746


In [232]:
df_model_comparsion = df_comparison.groupby(['model_name'])[['train_cagr_better','train_sharpe_ratio_better','calmar_better']].agg(lambda x: x.sum()/x.count())
df_model_comparsion

,train_cagr_better,train_sharpe_ratio_better,calmar_better
model_name,,,
gb,0.525000,0.525000,0.459854
rf,0.650000,0.650000,0.613889
ridge,0.733333,0.733333,0.688889


In [256]:
df_indicator_comparison = df_comparison.groupby(['col'])[['train_cagr_better','train_sharpe_ratio_better','calmar_better']].agg(lambda x: x.sum()/x.count())
df_indicator_comparison.sort_values(['train_sharpe_ratio_better','calmar_better'],ascending=False)

,train_cagr_better,train_sharpe_ratio_better,calmar_better
col,,,
PR_INT_20,0.866667,0.866667,0.826923
PR_INT_0,0.833333,0.833333,0.796296
CMMA_10_252,0.783333,0.783333,0.803571
MACD_10_100_5,0.750000,0.750000,0.740741
DT_RSI_2_20,0.700000,0.700000,0.719298
STO_20_1,0.700000,0.700000,0.589286
RSI_20,0.683333,0.683333,0.703704
LINTRND_10,0.683333,0.683333,0.696429
RSI_25,0.683333,0.683333,0.555556


## Timothy VS Talib on CAGR

Here we consider RSI timothy VS talib RSI

In [241]:
df_rsi = df_model[df_model['col'].isin(['RSI_20', 'RSI_20_ta','RSI_25', 'RSI_25_ta'])]
df_rsi = df_rsi[['ticker_name','col','train_cagr','train_sharpe_ratio','calmar']].set_index(['ticker_name','col'])\
    .groupby(level=['col','ticker_name']) \
    .apply(lambda g: g.apply('mean')) \
    .unstack(level='col')\
    .reset_index()
df_rsi.columns = ['_'.join(i).rstrip('_') for i in df_rsi.columns]
df_rsi.head(5)

,ticker_name,train_cagr_RSI_20,train_cagr_RSI_20_ta,train_cagr_RSI_25,train_cagr_RSI_25_ta,train_sharpe_ratio_RSI_20,train_sharpe_ratio_RSI_20_ta,train_sharpe_ratio_RSI_25,train_sharpe_ratio_RSI_25_ta,calmar_RSI_20,calmar_RSI_20_ta,calmar_RSI_25,calmar_RSI_25_ta
0,AAPL,13.540621,12.727101,9.668188,6.377368,9.792244,9.446179,7.685446,6.672483,244.015751,14291.940380,21.481496,112.274926
1,AMZN,17.150277,12.859222,17.219111,11.328701,9.683308,8.119666,9.680010,7.270788,18830.013415,110525.724317,1713.563326,1573.433821
2,BA,4.891481,10.269657,10.291296,10.156976,4.233492,5.352011,5.453233,5.332874,40.761994,38174.449777,2326.491223,3046.890757
3,BAC,19.396165,6.459145,14.853881,14.498492,8.552355,4.861147,6.832834,6.386023,23088.603343,62.587578,1951.355365,2331.056824
4,CAT,11.510977,11.028667,9.091717,5.206778,8.584977,8.385488,7.849769,5.538744,28427.384817,24.199246,149.144256,77.181846


In [252]:
timothy_columns = ['train_cagr_RSI_20', 
       'train_cagr_RSI_25', 
       'train_sharpe_ratio_RSI_20', 
       'train_sharpe_ratio_RSI_25']
rsi_result_dict = []
for i in list(range(len(timothy_columns))):
    tim_colname = timothy_columns[i]
    ta_colname = tim_colname + '_ta'
    df_rsi[tim_colname + '_better'] = (df_rsi[tim_colname] > df_rsi[ta_colname]).astype(int)
    df_rsi[tim_colname + '_ratio'] = (df_rsi[tim_colname] / df_rsi[ta_colname]).astype(float)

    rsi_result_dict.append({'indicator': tim_colname, '# tim > ta': np.sum(df_rsi[tim_colname + '_better']),
    '% tim > ta to entire cases': np.sum(df_rsi[tim_colname + '_better'])  /df_rsi.shape[0],
    'avg % tim / ta': round((np.mean(df_rsi[tim_colname + '_ratio']) * 100),2)
     })

pd.DataFrame(rsi_result_dict)


,indicator,# tim > ta,% tim > ta to entire cases,avg % tim / ta
0,train_cagr_RSI_20,15,0.75,114.66
1,train_cagr_RSI_25,17,0.85,129.34
2,train_sharpe_ratio_RSI_20,15,0.75,109.51
3,train_sharpe_ratio_RSI_25,18,0.90,118.85


## Ticker selection
using CAGR and sharpe ratio

In [327]:

selecting_ticker_for_model_explanation = df_model.groupby(['ticker_name','model_name']).agg({'train_cagr':'mean','train_sharpe_ratio':'mean','test_spearmanr':'mean'})
selecting_ticker_for_model_explanation = selecting_ticker_for_model_explanation.reset_index()
selecting_ticker_for_model_explanation.sort_values('train_cagr',ascending=False).head(3)
# selecting_ticker_for_model_explanation
## we select TSLA & BAC because is has craziest CAGR(most profitable)

,ticker_name,model_name,train_cagr,train_sharpe_ratio,test_spearmanr
54,TSLA,gb,322.472658,12.699307,0.047438
55,TSLA,rf,54.978725,6.886755,0.055246
9,BAC,gb,32.146260,12.695315,0.055011


In [325]:
selecting_ticker_for_model_explanation.sort_values('train_cagr',ascending=True).head(2)


,ticker_name,model_name,train_cagr,train_sharpe_ratio,test_spearmanr
53,TLT,ridge,0.211160,1.453426,0.032145
59,WMT,ridge,0.308024,1.509272,0.043247


In [323]:
selecting_ticker_for_model_explanation.sort_values('train_sharpe_ratio',ascending=False).head(2)
# selecting_ticker_for_model_explanation[selecting_ticker_for_model_explanation['ticker_name'].isin(['RUT','DJI',"OEX","NDX"])].sort_values('train_sharpe_ratio',ascending=False).head(5)
## need some index to model

,ticker_name,model_name,train_cagr,train_sharpe_ratio,test_spearmanr
45,RUT,gb,8.630551,15.715006,0.125077
51,TLT,gb,3.589642,15.606636,0.025742


In [324]:
selecting_ticker_for_model_explanation.sort_values('train_sharpe_ratio',ascending=True).head(2)


,ticker_name,model_name,train_cagr,train_sharpe_ratio,test_spearmanr
8,BA,ridge,0.57505,1.355835,0.061870
53,TLT,ridge,0.21116,1.453426,0.032145


In [ ]:
### hence, we select TSLA, BAC
## we also add RUT due to RUT has the highest performance among other index

In [328]:
## create some dictionary that for the model_explanation.py to use

explanable_list = []

for ticker in ['TLT','BA'] :#'TSLA','BAC','RUT']:
    df_temp = df_model[df_model['ticker_name'] == ticker]
    explanable_list.append(df_temp.loc[ ( df_temp.groupby(['model_name'])['train_cagr_rank'].idxmax() )])
    explanable_list.append(df_temp.loc[ ( df_temp.groupby(['model_name'])['train_cagr_rank'].idxmin() )])
    explanable_list.append(df_temp[df_temp['col'] == 'base'])

df_explain = pd.concat(
    explanable_list
)[['col','model_name','ticker_name']]

df_explain.to_dict(orient='records')

[{'col': 'base', 'model_name': 'gb', 'ticker_name': 'TLT'},
 {'col': 'RSI_20_ta', 'model_name': 'rf', 'ticker_name': 'TLT'},
 {'col': 'FTI_BF', 'model_name': 'ridge', 'ticker_name': 'TLT'},
 {'col': 'RSI_20_ta', 'model_name': 'gb', 'ticker_name': 'TLT'},
 {'col': 'PR_INT_20', 'model_name': 'rf', 'ticker_name': 'TLT'},
 {'col': 'PR_INT_0', 'model_name': 'ridge', 'ticker_name': 'TLT'},
 {'col': 'base', 'model_name': 'ridge', 'ticker_name': 'TLT'},
 {'col': 'base', 'model_name': 'rf', 'ticker_name': 'TLT'},
 {'col': 'base', 'model_name': 'gb', 'ticker_name': 'TLT'},
 {'col': 'DT_RSI_2_20', 'model_name': 'gb', 'ticker_name': 'BA'},
 {'col': 'ENT_4_16', 'model_name': 'rf', 'ticker_name': 'BA'},
 {'col': 'RSI_25', 'model_name': 'ridge', 'ticker_name': 'BA'},
 {'col': 'MADIFF_10_100', 'model_name': 'gb', 'ticker_name': 'BA'},
 {'col': 'CMMA_10_252', 'model_name': 'rf', 'ticker_name': 'BA'},
 {'col': 'PR_INT_0', 'model_name': 'ridge', 'ticker_name': 'BA'},
 {'col': 'base', 'model_name': 'ridge

In [333]:
df_explain = df_model[['col','model_name','ticker_name','test_rmse','test_spearmanr']]
df_explain = df_explain[df_explain['ticker_name'].isin(['TSLA','TLT','RUT','BA'])]
df_explain.groupby(['ticker_name','model_name']).agg({'test_rmse':'mean','test_spearmanr':'mean'})

test_rmse  test_spearmanr
ticker_name model_name                           
BA          gb          -0.021825        0.025586
            rf          -0.020963        0.027572
            ridge       -0.020439        0.061870
RUT         gb          -0.012042        0.125077
            rf          -0.011546        0.134699
            ridge       -0.011524        0.115135
TLT         gb          -0.008698        0.025742
            rf          -0.008277        0.027044
            ridge       -0.008230        0.032145
TSLA        gb          -0.039440        0.047438
            rf          -0.036716        0.055246
            ridge       -0.036093        0.019279